In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import joblib
import serial

app = Flask(__name__)
model = joblib.load('maternal.pkl')

ser = serial.Serial()
ser.port = 'COM7'
ser.baudrate = 9600
ser.bytesize = 8
ser.parity = serial.PARITY_NONE
ser.stopbits = serial.STOPBITS_ONE

def serialget():
    value=[]
    print('serialget')
    ser.open()
    v=b'A'
    ser.write(v)
    print('data sent')
    while True:
        for line in ser.read():
            if chr(line) != '$':
                value.append(chr(line))
            else:
                print("end")
                ser.close()
                return value

@app.route('/request1')
def request1():
    str1=''
    val=[]
    va=serialget()
    print(va)
    for v in va:
        #print('request for')
        if(v=='*'):
            continue
        else:  
            if(v!='#'): 
                str1+=v
                #print(str1)
            else:
                print(str1)
                val.append(float(str1))
                str1=""  
    print(val)
    return render_template('index.html',val1=val[0],val2=val[1])

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [(x) for x in request.form.values()]
    final_features = [np.array(int_features).astype(float)]
    print(final_features)
    prediction = model.predict(final_features)
    print(prediction)
    output = prediction[0]
    if output == 0:
        return render_template('index.html', prediction_text='High Risk')
    elif output == 1:
        return render_template('index.html', prediction_text='Low Risk')
    elif output == 2:
        return render_template('index.html', prediction_text='Mid Risk')
    print(output)

if __name__ == "__main__":
    app.run(host="localhost", port=7000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7000/ (Press CTRL+C to quit)
